## Pip installs

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install pysam

In [ ]:
!pip install HTSeq

In [5]:
!pip install enformer-pytorch>=0.5

In [3]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.8 MB/s eta 0:00:0000:0100:01


## Load modules and variable declarations

In [1]:
import torch
import polars as pl
from enformer_pytorch import Enformer, GenomeIntervalDataset
from datasets import concatenate_datasets, load_dataset
import os
import numpy as np
import pandas as pd
import pysam

/home/groups/cgawad/conda_for_pf_notebook/miniconda3_for_pf/envs/pf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tsv_dir= "/scratch/users/sschulz/pta_on_normal"
bed_path = "/scratch/users/sschulz/pta_on_normal/chr10.bed"


In [5]:
tsv_path = tsv_dir + "/CARTPt04_Scan2_svc_merged_extract_snp.hg38_multianno.tsv"

In [6]:
tsv = pd.read_table(tsv_path, sep='\t')

In [7]:
tsv['CHROM'][0]

'chr1'

In [23]:
bam_path=tsv_dir + '/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-D4_S26.realigned_deduped_sorted.bam'

## Function and class definitions

In [72]:
def makeLlamaDataset(tsv_dir, bam_path, bed_path):
    '''
        from a directory containing many annotated tsv files and a bed path, create a huggingface dataset for use in llama
        
        start by just passing lines from vcf to llama for fine tuning, along with a line that says 
        "The read/basepairs/sequence at this position is:
        The read information from reference is:"
        
        This is a pretty brute force way to do it but maybe it'll create something coherent from llama.

    '''
    for filename in os.listdir(tsv_dir):
        if filename.endswith('tsv'):
            tsv_file = os.path.join(tsv_dir, filename)
            tsv_length=len(tsv_file)
            counter = 0
            print(tsv_file)
            for i in range(tsv_length):
                chrom = tsv['CHROM'][i]
                start_pos = tsv['POS'][i]
                sample = tsv['SAMPLE'][i]
                gt = tsv['GT'][i]
                if gt == '0/1' or gt == '1/1':
                    print(sample, gt)
                    samfile = pysam.AlignmentFile(bam_path, "rb" )
                    pileup = samfile.pileup(chrom, start_pos, start_pos+1, min_mapping_quality=58)
                    first = pileup
                    for x in first:
                        print(str(x))

                # for x in pileup:
                #     if counter == 0:
                #         print(str(x))

            

In [73]:
makeLlamaDataset(tsv_dir, bam_path, bed_path)

/scratch/users/sschulz/pta_on_normal/CARTPt04_Scan2_svc_merged_extract_snp.hg38_multianno.tsv
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A8_S50 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E10_S68 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F6_S41 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A8_S50 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B4_S24 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E10_S68 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E11_S75 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F11_S76 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F6_S41 0/1
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51 0/1
0	873536	1
VH00736:15:AAAVYTKHV:2:1212:61442:31593	99	#0	873537	60	100M	#0	873668	231	GTGTCAGACACTGTGGTGGAGCCCTACAACGCCACCCTCTCAGTCCACCAGCTCATAGAAAATGTGGATGAGACCTTCTGCATAGATAACGAAGCGCTAT	array('B', [34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 3